In [ ]:
#
# Project:
#      PyTorch Dojo (https://github.com/wo3kie/ml-dojo)
#
# Author:
#      Lukasz Czerwinski (https://www.lukaszczerwinski.pl/)
#

$$
\begin{pmatrix}
y_0 \\
y_1 \\
\vdots \\
y_S \\
\end{pmatrix}

=

\begin{pmatrix}
x_{00} & x_{01} & \cdots & x_{0F} \\
x_{10} & x_{11} & \cdots & x_{1F} \\
\vdots & \vdots & \ddots & \vdots \\
x_{S0} & x_{S1} & \cdots & x_{SF} \\
\end{pmatrix} 

\cdot

\begin{pmatrix}
w_0 \\
w_1 \\
\vdots \\
w_F \\
\end{pmatrix}

+

\begin{pmatrix}
b \\
b \\
\vdots \\
b \\
\end{pmatrix}
$$


In [ ]:
from torch import randn, Size
from torch.nn import Linear, MSELoss
from torch.optim import SGD

# %run common.ipynb
import import_ipynb
from common import equal, T # type: ignore


def linear_regression_SGD_autograd(X, y, epochs=1000, lr=0.02):
    (S, F) = X.shape

    model = Linear(in_features=F, out_features=1, bias=True)
    w = model.weight
    assert w.shape == Size([1, F])

    b = model.bias
    assert b.shape == Size([1])

    optimizer = SGD(model.parameters(), lr=lr)

    for _ in range(epochs):
        # dL_dW = 0
        # dL_db = 0
        optimizer.zero_grad()
        
        # predicted = X @ w + b
        predicted = model(X)

        # error = predicted - y
        # loss = 1/S * sum(error ** 2)
        loss = MSELoss()(predicted, y)

        # dL_dW += (2/S) * x.T * error
        # dL/db += 2/S * sum(error)
        loss.backward()

        # w = w - lr * dL_dW
        # b = b - lr * dL_db
        optimizer.step()

    return (loss, w, b)


def _test_linear_regression_SGD_autograd(S, W, B, epochs=2000, lr=0.01):
    """
    Tests the linear regression using Stochastic Gradient Descent (SGD) with manual gradient calculation, 
    by generating synthetic data with known weights, and verifies that the computed weights and bias are correct.

    Parameters:
        S (float): Samples
        W (float): Model's weight(s)
        B (float): Model's bias
    """

    F = W.shape[0]
    x = randn(S, F)
    assert(x.shape == Size([S, F]))

    y = x @ W + B
    assert(y.shape == Size([S, 1]))

    loss, w, b = linear_regression_SGD_autograd(x, y, epochs, lr)
    
    # torch.nn.Linear weights are stored in row vector, but we so we need to reshape it to column vector.
    w = w.reshape(W.shape)

    assert(equal(loss, 0.0))
    assert(equal(b, B))
    assert(equal(w, W))


def test_linear_regression_SGD_autograd():
    # 1 out of 10 times the test fails due to random initialization of weights and bias, 
    # which can lead to non-convergence in some cases. Simply re-run the test to pass it.

    _test_linear_regression_SGD_autograd(10, W=T([[0.1]]), B=0.2)
    _test_linear_regression_SGD_autograd(10, W=T([[0.3], [0.4]]), B=0.5)
    _test_linear_regression_SGD_autograd(10, W=T([[0.6], [0.7], [0.8]]), B=0.9)


if __name__ == "__main__":
    test_linear_regression_SGD_autograd()
